![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a V4L2 device and display the output on a monitor using a DRM/KMS display device. This notebook uses the GStreamer multimedia framework.

Two types of V4L2 devices are supported in this notebook:
* Virtual Video Test driver (vivid)
* USB Video Class driver (UVC)

See nb2 for details on the mentioned V4L2 devices.

The display device is uses the Xilinx DRM/KMS driver. A video mixer supports alpha blending of multiple layers (also called planes). The plane formats are fixed and configured as follows:
* 4 RGB planes (IDs: 30-33)
* 4 YUY2 planes (IDs: 34-37)
* 1 ARGB plane (ID: 38) - this is the primary plane used for setting the CRTC resolution

The video mixer is connected to an HDMI encoder which drives the display. Both video mixer and HDMI encoder are implemented inside the FPGA.

The video pipeline is composed of the following GStreamer elements:
* The ``v4l2src`` element is used to capture video from a V4L2 device
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device and displays the video on a monitor using DRM/KMS.
2. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import pydot
import sys
import time
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the Base TRD notebook 3 (nb3).

In [ ]:
nb = "nb3"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environement variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Optionally enable debug (default off) and set the debug level.

In [ ]:
Gst.init(None)
Gst.debug_set_active(False)
Gst.debug_set_default_threshold(1)

# 3. Create and Configure the GStreamer Elements

Create the ``v4l2src`` element and set some properties:
* Set the ``io_mode`` to ``mmap`` for mapping device memory into application address space.
* Set the ``device`` property to the desired video device node

The default device node is set to ``/dev/video0`` which corresponds to vivid. Update the value below next to the comment to match the video node of your USB webcam as per above instructions.

**Note:** Refer to nb2 for instructions on how to discover v4l2 devices.

In [ ]:
io_mode = "mmap"
device = "/dev/video0" # Change the video device node here

src = Gst.ElementFactory.make("v4l2src")
src.set_property("io-mode", io_mode)
src.set_property("device", device)

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

The default resolution is set to 1280x720 and the format to YUY2 as those are commonly supported by USB webcams.

In [ ]:
width = 1280
height = 720
fmt = "YUY2"

caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string("video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt)
caps.set_property("caps", cap)

The display driver creates a DRM device node with the module name ``xlnx``.

List information about the DRM device by passing the module name to the ``modeprint`` utility.

In [ ]:
!modeprint xlnx

Create the ``kmssink`` element and set some properties:
* Set the ``driver-name`` property to the Xilinx DRM driver name ``xlnx``.
* Set the ``plane-id`` property to the ID value of the target plane. The default value 34 is set to the first YUY2 plane.
* Set the ``fullscreen-overlay`` property to ``False`` to keep the CRTC set to the native display resolution.
* Set the ``render-rectangle`` property to a quadruple consisting of x-offset, y-offset, width, and height. The render-rectangle allows moving a plane position on the display.

In [ ]:
driver_name = "xlnx"
plane_id = 34
xoff = 0 # Change this value to move the plane position in the x-direction
yoff = 0 # Change this value to move the plane position in the y-direction
fullscreen_overlay = False
render_rectangle = Gst.ValueArray((xoff, yoff, width, height))

sink = Gst.ElementFactory.make("kmssink")
sink.set_property("driver-name", driver_name)
sink.set_property("plane-id", plane_id)
sink.set_property("fullscreen-overlay", fullscreen_overlay)
sink.set_property("render-rectangle", render_rectangle)

# Uncomment the below code to read back the newly set property values
#print("sink properties: ")
#print("driver-name: " + str(sink.get_property("driver-name")))
#print("plane-id: " + str(sink.get_property("plane-id")))
#print("fullscreen-overlay: " + str(sink.get_property("fullscreen-overlay")))

# 4. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(sink)

src.link(caps)
caps.link(sink);

The ``bus_call`` function listens on the bus for ``EOS`` and ``ERROR`` events. If any of these events occur, stop the pipeline (set to ``NULL`` state) and quit the main loop.

In case of an ``ERROR`` event, parse and print the error message.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. 

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 5. View the GStreamer Pipeline Graph

Register dot plugins for png export to work.

In [ ]:
!dot -c

Convert the dot file to png and display the pipeline graph. The image will be displayed below the following code cell. Double click on the generate image file to zoom in.

**Note:** This step may take a few seconds.

In [ ]:
dotfile = dotdir + "/" + nb + ".dot"
graph = pydot.graph_from_dot_file(dotfile, 'utf-8')
display(Image(graph[0].create(None, 'png', 'utf-8')))

# 6. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and display it on a monitor
2. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>